In [1]:
knitr::opts_chunk$set(echo = TRUE)
pacman::p_load(pacman, tidyverse, lattice, ggpubr, ggrepel, svglite, see) # Visual.
pacman::p_load(lme4, merTools, lmtest, lmerTest, lcmm, performance) # Modeling.
pacman::p_load(jsonlite) # IO; for Python Dict (json).

dir_input <- file.path("input")
dir_TEMP <- file.path("TEMP") # Intermediate files.
dir_dict <- file.path("dicts") # Look up dictionaries such as paper2meta; main data directory.
dir_output <- file.path("output") # Figures.
dir_DEBUG <- file.path("DEBUG")

# HELPER FUNCTIONS
make_empty_list_or_else <- function(x) {
    # x is either char vector or list, and if list, it can be empty (length(x)==0), in which case NA.
    if (length(x) == 0) {
        NA
    } else {
        x
    }
}

get_var_name <- function(x) {
    deparse(substitute(x))
}

get_slr_stats <- function(df, dv, iv, interval = "confidence", level = 0.95) {
    model.slr <- lm(as.formula(str_glue("{dv} ~ 1 + {iv}")), data = df, weights = inv_var_weight) # Weighted linear fit.
    df.pred <- as.data.frame(predict(model.slr, newdata = dplyr::select(df, all_of(iv)), interval = interval, level = level))
    s <- summary(model.slr)
    fstat.val <- s$fstatistic[[1]]
    fstat.df1 <- s$fstatistic[[2]]
    fstat.df2 <- s$fstatistic[[3]]
    fstat.pval <- pf(fstat.val, fstat.df1, fstat.df2, lower.tail = FALSE)
    adj.r.sq <- s$adj.r.squared
    txt <- bquote(
        F(.(fstat.df1),.(fstat.df2))==.(round(fstat.val, 3))~~p==.(round(fstat.pval, 3))~~{R[a]}^2==.(round(adj.r.sq, 3))
        )

    list("txt" = txt, "df.pred" = df.pred, "fstat.pval" = fstat.pval)
}

get_col_freq <- function(col, decreasing = TRUE) {
    sort(table(col), decreasing = decreasing)
}

In [2]:
# Aff meta.
df.department.meta <- readRDS(file.path(dir_dict, "department2meta.Rds"))

# Citation pair table.
df.cite <- readRDS(file.path(dir_dict, "cite2author_aff.Rds"))
# fromJSON converts list a into b, where b[[1]] is the list.
# If a = [4,2], then b[[1]] is [4,2], but if a = [], then b[[1]] is list() (empty list in R).
# in printed data.frame, this empty list may show up as NULL, but it's not NULL.
df.cite$last_author_department <- lapply(df.cite$last_author_department, fromJSON)
df.cite$last_author_department <- lapply(df.cite$last_author_department, make_empty_list_or_else)

# Convert rowname into column called "index".
df.department.meta <- tibble::rownames_to_column(df.department.meta, var = "index")
df.cite <- tibble::rownames_to_column(df.cite, var = "index")
df.cite$index <- as.integer(df.cite$index) # cite2SRPC_ms order is int.
df.cite$citing_gender <- as.factor(df.cite$citing_gender)
df.cite$cited_gender <- factor(df.cite$cited_gender, levels = levels(df.cite$citing_gender))


# Unnest list-columns, such that each element in the list-column gets a separate row.
# Only do it on demand, as in if we group by department, we unnest department col only.
# Because if we do both, we will get Cartesian product, which means duplicates when we group by only one col.
df.cite <- tidyr::unnest_longer(df.cite, col = last_author_department, values_to = "department", keep_empty = TRUE)

# Drop rows where any column is NA.
df.cite <- tidyr::drop_na(df.cite)

df.cite$department <- as.factor(df.cite$department) # Convert from char to factor.
df.department.meta$index <- factor(df.department.meta$index, levels = levels(df.cite$department))

# Add department measure (synthesis) to the table.
df.cite <- dplyr::left_join(df.cite, dplyr::select(df.department.meta, c("index", "synthesis")), by = c("department" = "index")) %>%
    dplyr::rename(citing_synthesis = synthesis)

# df.cite <- dplyr::select(df.cite, -c("department"))  # Remove department col since we only need its measure.

# Drop rows where any column is NA. In this line, the synthesis has NA in math because math is removed from 28 departments due to not meeting sample size requirement.
df.cite <- tidyr::drop_na(df.cite)


In [3]:
get_ttest <- function(fit, term) {
    ct <- summary(fit)$coefficients
    effect.size <- ct[term, "Estimate"]
    ci.95 <- confint(fit)[term, ] # 95% Wald CI
    t.stat <- ct[term, "t value"]
    p.val <- ct[term, "Pr(>|t|)"]
    dof <- fit$df.residual

    sprintf(
        "slope = %.4f 95%% CI (%.4f, %.4f), t(%d) = %.4f, p = %.4f",
        effect.size, ci.95[1], ci.95[2], dof, t.stat, p.val
    )
}

In [4]:
# Multiple linear regression.

fit.neg <- lm("y_neg ~ 1 + hindex_abs + citing_gender + citing_gender:cited_gender + citing_synthesis + citing_gender:hindex_abs + citing_gender:citing_synthesis", data = df.cite, weights=wy_neg)
fit.pos <- lm("y_pos ~ 1 + hindex_abs + citing_gender + citing_gender:cited_gender + citing_synthesis + citing_gender:hindex_abs + citing_gender:citing_synthesis", data = df.cite, weights=wy_pos)


In [ ]:
(summary(fit.neg))

(summary(fit.pos))


In [ ]:
get_ttest(fit.neg, "hindex_abs:citing_genderwoman")
get_ttest(fit.neg, "citing_genderwoman:citing_synthesis")
get_ttest(fit.pos, "hindex_abs:citing_genderwoman")
get_ttest(fit.pos, "citing_genderwoman:citing_synthesis")

In [7]:
fit.neg.mainonly <- lm("y_neg ~ 1 + hindex_abs + citing_gender + citing_synthesis", data = df.cite, weights = wy_neg)
fit.pos.mainonly <- lm("y_pos ~ 1 + hindex_abs + citing_gender + citing_synthesis", data = df.cite, weights = wy_pos)


In [ ]:
performance::check_collinearity(fit.neg.mainonly)

In [ ]:
performance::check_collinearity(fit.pos.mainonly)